<a href="https://colab.research.google.com/github/TurkuNLP/textual-data-analysis-course/blob/main/dspy_ocr_correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DSPy

* A small demonstration on the OCR post-correction task
* On a dataset of 18th century English
* Main metric: character error rate
* Absolute or relative (relative measures improvement compared to baseline)

# Config DSPy

In [ ]:
import dspy
import unicodedata
import re
import json
import random

#api_keys.py has a GPT4o_API_KEY variable
from api_keys import *

lm = dspy.LM('openai/gpt-4o-mini', api_key=GPT4o_API_KEY)
dspy.configure(lm=lm)

# Evaluation

* Normalize the text (not to get overwhelmed by whitespace and newlines)
* CER is a reasonable character error rate
* relative rate is often used

In [ ]:
def norm(s):
    """Basic text normalization function, removes unicode crud, also newlines etc."""
    s=unicodedata.normalize("NFKC",s)
    s=re.sub(r"\s+"," ",s)
    return s

def eval_cer(reference,system_out):
    """Character error rate evaluation."""
    import evaluate #This *must* be loaded here because of parallel processing, otherwise you get nasty crashes, took me forever to figure
    cer_metric=evaluate.load("cer")

    # We need to avoid any danger here, so I make damn sure I never return anything <=0 or >1
    if len(reference)<10 or len(system_out)<10:
        m_val=0.001 #Too short, I return some safety nonzero value
    else:
        try:
            c=cer_metric.compute(predictions=[system_out],references=[reference])
        except:
            c=0.001 #...safety value in case something crashes
        c=min(c,1.0)
        c=max(c,0.001) #force c into (0,1] interval
        m_val=(1.0-c) #Make c be "bigger is better" (CER is smaller is better)
    return m_val

def eval_cer_absolute(reference,system_out,trace=None):
    m_val=eval_cer(reference.denoised_text,system_out.denoised_text)
    if trace is None:
        return m_val #used in eval, should return a float
    else:
        return m_val>0.9 #used to select few shot examples, should return True/False

def eval_cer_relative(reference,system_out,trace=None):
    m_val=eval_cer(reference.denoised_text,system_out.denoised_text) #what did we get?
    ref_val=eval_cer(reference.denoised_text,reference.noised_text) #what was there to be had?
    #print("M:",m_val,"R:",ref_val)
    relative_val=(m_val-ref_val)/(1-ref_val)
    relative_val=min(1.0,relative_val)
    relative_val=max(0.001,relative_val)
    if trace is None:
        return relative_val
    else:
        return relative_val > 0.2 #let's call the example good if it fixes more than 20% of what was fixable

# Load data

* DSPy has own classes for data
* `Example(field1=...,field2=...,field3=...).with_inputs(["field1","field3",...])`

In [ ]:
def read_data(fname):
    """OCR denoise data comes as dictionaries with "input" and "output" keys"""
    with open(fname) as f:
        samples=[json.loads(l) for l in f.readlines()]
        samples=[dspy.Example(noised_text=norm(s["input"]), denoised_text=norm(s["output"])).with_inputs("noised_text") for s in samples]
    random.shuffle(samples)
    return samples

samples=read_data("/home/ginter/ocr-postcorrection-data/English/SEGMENTED_DOCUMENTS/en_dev_sample200.jsonl")

# Simplest DSPy program possible

In [ ]:
program=dspy.Predict("noised_text: str -> denoised_text: str")

In [ ]:
print("**** Evaluate the data ****")
ev=dspy.evaluate.Evaluate(devset=samples[:10],display_progress=True,display_table=5)
ev(program,metric=eval_cer_relative)

**** Evaluate the data ****
M: 0.8442211055276382 R: 0.8726968174204355                                                                                                                                                                           | 0/10 [00:00<?, ?it/s]
M: 0.9901960784313726 R: 0.9786096256684492████████████▋                                                                                                                                                      | 1/10 [00:02<00:23,  2.63s/it]
M: 0.9304029304029304 R: 0.9496336996336996█████████████████████████████▍                                                                                                                                     | 2/10 [00:05<00:22,  2.77s/it]
M: 0.9758364312267658 R: 0.9628252788104089██████████████████████████████████████████████                                                                                                                     | 3/10 [00:07<00:18,  2.61s/it]
M: 0.991142604074402

2025/02/22 22:42:41 INFO dspy.evaluate.evaluate: Average Metric: 2.8335946806816383 / 10 (28.3%)


,noised_text,example_denoised_text,pred_denoised_text,eval_cer_relative
0,"of Bothwel; 2dly, toThomas x489' lord Erskinc, ancellor of the ear...","of Bothwel; 2dly,Ibid. ad. ann. 1489. to Thomas lord Erskine, ance...","of Bothwell; secondly, to Thomas lord Erskine, chancellor of the e...",✔️ [0.001]
1,"observing that, although he feels it impofiible to marry the lady ...","observing that, although he feels it impossible to marry the lady ...","observing that, although he feels it impossible to marry the lady ...",✔️ [0.542]
2,"C 4 3 ) EngliJ, St. George's Eve; all which fignfy one and the fam...","English, St. George's Eve; all which signify one and the same thin...","C 4 3) English, St. George's Eve; all of which signify one and the...",✔️ [0.001]
3,"Till left her lelplefi fate to moarre, rTglkded, lo-aing andforlor...","Till left her helpless state to mourn, Neglected, loving and forlo...","Till left her helpless fate to mourn, troubled, longing and forlor...",✔️ [0.350]
4,"wheeled upon the rear of the pursuers, and made great havock among...","wheeled upon the rear of the pursuers, and made great havock among...","wheeled upon the rear of the pursuers, and made great havoc among ...",✔️ [0.565]


28.34

In [ ]:
teleprompter = dspy.teleprompt.MIPROv2(metric=eval_cer_relative,auto="medium")
program=dspy.Predict("noised_text: str -> denoised_text: str")
optimized_program = teleprompter.compile(program.deepcopy(),trainset=samples[:75],max_bootstrapped_demos=5,max_labeled_demos=5)
optimized_program.save("./mp_optimized_v4/", save_program=True)

2025/02/22 22:48:06 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: True
num_candidates: 19
valset size: 60



Projected Language Model (LM) Calls

Based on the parameters you have set, the maximum number of LM calls is projected as follows:

- Prompt Generation: 10 data summarizer calls + 19 * 1 lm calls in program + (2) lm calls in program-aware proposer = 31 prompt model calls
- Program Evaluation: 25 examples in minibatch * 25 batches + 60 examples in val set * 3 full evals = 805 LM Program calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of program calls * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectat

2025/02/22 22:48:13 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/02/22 22:48:13 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/02/22 22:48:13 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=19 sets of demonstrations...


Bootstrapping set 1/19
Bootstrapping set 2/19
Bootstrapping set 3/19


  7%|█████████████▍                                                                                                                                                                                           | 1/15 [00:09<02:18,  9.88s/it]

...

2025/02/22 23:11:03 INFO dspy.evaluate.evaluate: Average Metric: 6.981523318121285 / 25 (27.9%)
2025/02/22 23:11:03 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 27.93 on minibatch of size 25 with parameters ['Predictor 0: Instruction 18', 'Predictor 0: Few-Shot Set 15'].
2025/02/22 23:11:03 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [20.89, 29.74, 23.78, 32.09, 15.86, 30.48, 29.98, 27.71, 32.62, 27.09, 40.49, 28.29, 24.91, 35.34, 32.72, 33.29, 43.11, 47.18, 23.12, 34.17, 36.84, 20.63, 27.93]
2025/02/22 23:11:03 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [16.42, 35.39, 39.24]
2025/02/22 23:11:03 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 39.24
2025/02/22 23:11:03 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/02/22 23:11:03 INFO dspy.teleprompt.mipro_optimizer_v2: == Trial 27 / 28 - Minibatch ==



M: 0.7660194174757282 R: 0.5970873786407767                                                                                                                                                                           | 0/25 [00:00<?, ?it/s]
M: 0.9656387665198238 R: 0.9841409691629956██▋                                                                                                                                                                | 1/25 [00:05<02:10,  5.45s/it]
M: 0.9505637467476149 R: 0.9505637467476149█████████▎                                                                                                                                                         | 2/25 [00:05<00:55,  2.40s/it]
M: 0.9791855203619909 R: 0.96289592760181██████████████████                                                                                                                                                   | 3/25 [00:05<00:31,  1.43s/it]
M: 0.9448209099709584 R: 0.9225556631171346████

2025/02/22 23:11:31 INFO dspy.evaluate.evaluate: Average Metric: 6.791320628735322 / 25 (27.2%)
2025/02/22 23:11:31 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 27.17 on minibatch of size 25 with parameters ['Predictor 0: Instruction 1', 'Predictor 0: Few-Shot Set 11'].
2025/02/22 23:11:31 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [20.89, 29.74, 23.78, 32.09, 15.86, 30.48, 29.98, 27.71, 32.62, 27.09, 40.49, 28.29, 24.91, 35.34, 32.72, 33.29, 43.11, 47.18, 23.12, 34.17, 36.84, 20.63, 27.93, 27.17]
2025/02/22 23:11:31 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [16.42, 35.39, 39.24]
2025/02/22 23:11:31 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 39.24
2025/02/22 23:11:31 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/02/22 23:11:31 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 28 / 28 - Full Evaluation =====
2025/02/22 23:11:31 INFO dspy.teleprompt.mipro_optimizer_v2: 


M: 0.9233971690258118 R: 0.8834304746044963                                                                                                                                                                           | 0/60 [00:00<?, ?it/s]
M: 0.991321118611379 R: 0.9729990356798457                                                                                                                                                                    | 1/60 [00:02<02:20,  2.39s/it]
M: 0.7611650485436894 R: 0.5970873786407767                                                                                                                                                                   | 1/60 [00:02<02:20,  2.39s/it]
M: 0.9791855203619909 R: 0.96289592760181██████▎                                                                                                                                                              | 3/60 [00:02<00:37,  1.52it/s]
M: 0.9609035621198957 R: 0.9626411815812337████

2025/02/22 23:12:16 INFO dspy.evaluate.evaluate: Average Metric: 23.47015277352771 / 60 (39.1%)
2025/02/22 23:12:16 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [16.42, 35.39, 39.24, 39.12]
2025/02/22 23:12:16 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 39.24
2025/02/22 23:12:16 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/02/22 23:12:16 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/02/22 23:12:16 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 39.24!


In [ ]:
optimized_program

Predict(StringSignature(noised_text -> denoised_text
    instructions='In a world where historical and literary texts are at risk of being lost to time due to typographical errors and archaic spellings, your task is to act as a guardian of language. Given the `noised_text` that contains various distortions, your mission is to meticulously restore it to its original clarity and meaning. Produce the `denoised_text` that not only corrects the errors but also preserves the essence of the text, ensuring it is both accurate and comprehensible for future generations of scholars and enthusiasts alike.'
    noised_text = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Noised Text:', 'desc': '${noised_text}'})
    denoised_text = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Denoised Text:', 'desc': '${denoised_text}'})
))

In [ ]:
print("Initial",ev(program,metric=eval_cer_relative))
print("Improved",ev(optimized_program,metric=eval_cer_relative))


Average Metric: 2.83 / 10 (28.3%): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:27<00:00,  2.76s/it]

2025/02/22 23:17:58 INFO dspy.evaluate.evaluate: Average Metric: 2.8335946806816383 / 10 (28.3%)


,noised_text,example_denoised_text,pred_denoised_text,eval_cer_relative
0,"of Bothwel; 2dly, toThomas x489' lord Erskinc, ancellor of the ear...","of Bothwel; 2dly,Ibid. ad. ann. 1489. to Thomas lord Erskine, ance...","of Bothwell; secondly, to Thomas lord Erskine, chancellor of the e...",✔️ [0.001]
1,"observing that, although he feels it impofiible to marry the lady ...","observing that, although he feels it impossible to marry the lady ...","observing that, although he feels it impossible to marry the lady ...",✔️ [0.542]
2,"C 4 3 ) EngliJ, St. George's Eve; all which fignfy one and the fam...","English, St. George's Eve; all which signify one and the same thin...","C 4 3) English, St. George's Eve; all of which signify one and the...",✔️ [0.001]
3,"Till left her lelplefi fate to moarre, rTglkded, lo-aing andforlor...","Till left her helpless state to mourn, Neglected, loving and forlo...","Till left her helpless fate to mourn, troubled, longing and forlor...",✔️ [0.350]
4,"wheeled upon the rear of the pursuers, and made great havock among...","wheeled upon the rear of the pursuers, and made great havock among...","wheeled upon the rear of the pursuers, and made great havoc among ...",✔️ [0.565]


Initial 28.34
Average Metric: 5.54 / 10 (55.4%): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:01<00:00,  6.18s/it]

2025/02/22 23:19:00 INFO dspy.evaluate.evaluate: Average Metric: 5.536892423625517 / 10 (55.4%)


,noised_text,example_denoised_text,pred_denoised_text,eval_cer_relative
0,"of Bothwel; 2dly, toThomas x489' lord Erskinc, ancellor of the ear...","of Bothwel; 2dly,Ibid. ad. ann. 1489. to Thomas lord Erskine, ance...","of Bothwell; 2dly, to Thomas lord Erskine, chancellor of the earl ...",✔️ [0.336]
1,"observing that, although he feels it impofiible to marry the lady ...","observing that, although he feels it impossible to marry the lady ...","observing that, although he feels it impossible to marry the lady ...",✔️ [0.500]
2,"C 4 3 ) EngliJ, St. George's Eve; all which fignfy one and the fam...","English, St. George's Eve; all which signify one and the same thin...","English, St. George's Eve; all which signify one and the same thin...",✔️ [0.764]
3,"Till left her lelplefi fate to moarre, rTglkded, lo-aing andforlor...","Till left her helpless state to mourn, Neglected, loving and forlo...","Till left her helpless fate to mourn, troubled, longing and forlor...",✔️ [0.625]
4,"wheeled upon the rear of the pursuers, and made great havock among...","wheeled upon the rear of the pursuers, and made great havock among...","wheeled upon the rear of the pursuers, and made great havoc among ...",✔️ [0.565]


Improved 55.37


**Sweet! That worked!**

# Simpler optimizer

* Let's try an optimizer which only tries to come up with good few_shot examples

In [ ]:
optimizer = dspy.teleprompt.BootstrapFewShot(
    metric=eval_cer_relative,
    max_bootstrapped_demos=8,
    max_labeled_demos=0,
    max_rounds=10,
)
program=dspy.Predict("noised_text: str -> denoised_text: str")
optimized_program_bsfs = optimizer.compile(program.deepcopy(),trainset=samples[:75])
optimized_program_bsfs.save("./mp_optimized_BSFS_v1/", save_program=True)

  0%|                                                                                                                                                                                                                 | 0/75 [00:00<?, ?it/s]

Average Metric: 0.00 / 5 (0.0%):  80%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                 | 4/5 [1:08:32<17:08, 1028.14s/it]


 28%|████████████████████████████████████████████████████████                                                                                                                                                | 21/75 [13:30<34:44, 38.60s/it]

Bootstrapped 8 full traces after 21 examples for up to 10 rounds, amounting to 139 attempts.


In [ ]:

print(optimized_program_bsfs)
print("Improved",ev(optimized_program_bsfs,metric=eval_cer_relative))

Predict(StringSignature(noised_text -> denoised_text
    instructions='Given the fields `noised_text`, produce the fields `denoised_text`.'
    noised_text = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Noised Text:', 'desc': '${noised_text}'})
    denoised_text = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Denoised Text:', 'desc': '${denoised_text}'})
))
Average Metric: 3.73 / 10 (37.3%): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:07<00:00,  6.74s/it]

2025/02/23 00:01:57 INFO dspy.evaluate.evaluate: Average Metric: 3.728672425631721 / 10 (37.3%)


,noised_text,example_denoised_text,pred_denoised_text,eval_cer_relative
0,"of Bothwel; 2dly, toThomas x489' lord Erskinc, ancellor of the ear...","of Bothwel; 2dly,Ibid. ad. ann. 1489. to Thomas lord Erskine, ance...","of Bothwell; 2dly, to Thomas lord Erskine, chancellor of the earl ...",✔️ [0.001]
1,"observing that, although he feels it impofiible to marry the lady ...","observing that, although he feels it impossible to marry the lady ...","observing that, although he feels it impossible to marry the lady ...",✔️ [0.542]
2,"C 4 3 ) EngliJ, St. George's Eve; all which fignfy one and the fam...","English, St. George's Eve; all which signify one and the same thin...","C 4 3) English, St. George's Eve; all which signify one and the sa...",✔️ [0.291]
3,"Till left her lelplefi fate to moarre, rTglkded, lo-aing andforlor...","Till left her helpless state to mourn, Neglected, loving and forlo...","Till left her helpless fate to mourn, troubled, longing and forlor...",✔️ [0.350]
4,"wheeled upon the rear of the pursuers, and made great havock among...","wheeled upon the rear of the pursuers, and made great havock among...","wheeled upon the rear of the pursuers, and made great havoc among ...",✔️ [0.565]


Improved 37.29


In [ ]:
print(optimized_program_bsfs.demos)

[Example({'augmented': True, 'noised_text': 'observing that, although he feels it impofiible to marry the lady himself, he cannot endure the thought of her living for another. This new\'misfortune finks the father to the ground, upon which he is left to die on one fide of the fiage, while the lady flands flatue-ftruck with grief on the other. Neither of these objeets go to the heart of our hero. On the contrary, he intimates that there is no way left to pacify his fears on this curious point of delicacy, but the death of this beloved mistress. Hereupon the poet makes her obligingly take the hint by throwing herself into an attitude to receive the blow from the hand of her lover; who, how- ever, rather hesitates about it, upon which the lady presents her beautiful bosom (all heroines you know miiri be beautiful) to any of the robbers; none of whom can be found to Scar that whiter in of her\'s "Than monumental alabalter." when men, who live by pillage and murder, are thus tender-hearted,